In [1]:
import pandas as pd 

In [2]:
review_data1 = pd.read_csv('../../data/food_review_datas1.csv')
review_data2 = pd.read_csv('../../data/food_review_datas2.csv')
review_data3 = pd.read_csv('../../data/food_review_datas3.csv')

In [3]:
review_data = pd.concat([review_data1,review_data2,review_data3],axis=0,ignore_index=True)

In [4]:
review_data.drop('Unnamed: 0',axis=1,inplace=True)

In [5]:
review_data.drop_duplicates(subset='id',inplace=True)

In [6]:
review_data['review_datas'] = review_data['review_datas'].apply(lambda x: eval(x))

## 리뷰데이터 json구조
## 5개의 딕셔너리 
## data 0 : visitorReviews , 1: visitorReviews , 2: visitorReviewStats, 3: visitorReviewStats, 4: restaurant(fsasReviews) 블로그 리뷰  6: 정보

0. visitorReviews : 사진 리뷰들 20개
    - items(리스트)
        - id : 이거를 primarykey로 
        - rating : None 값 존재
        - body: 리뷰 텍스트 내용 (None 값 존재)
        - visited : 방문 날짜
        - votedKeywords(리스트로 존재)
            - 각 displayName을 키 값으로 True로 넣자

2. visitorReviews : 최신 후기 3개
    - items(리스트)3개 
        - id : 위와 동일
        - rating : "
        - body : "
        - visited : "

3. visitorReviewStats:
    - id : 식당 id 값
    - review(None 존재)
        - avgRating : 평균 별점 
        - TotalCount : 별점 부여한 총 리뷰
        - scores : 각 점수별 갯수 (문제는 각 점수 구간을 모름)
        - starDistribution : 점수 구간 
        - imgageReviewCount : 사진리뷰
        - authorCount : 작성자 숫자
        - maxSingleReviewScoreCount: 5점 부여한 리뷰 숫자
    - analysis(None 존재) : 리뷰 분석 key 
        - themes(리스트로 존재) : 리뷰에  언급된 themes들 
            - label을 키값으로 value는 count를 넣자
        - menus(리스트로 존재 ) :리뷰에 언급된 메뉴들:
            - label을 키 값으로 value에는 count를 넣자
        - votedKeyword : 리뷰 작성시 투표한 키워드들 
            - totalCount: 키워드에 투표한 총 수
            - reviewCount: 키워드를 체크한 리뷰총 개수
            - userCount: 키워드를 체크한 유저수? 유저별로 키워드를 중복 투표할 수 있으니깐?
            - details(리스트로 존재) : displayName 를 키값으로 count를 벨류로 넣자 
        - visitorReviewsTotal: 방문자 총리뷰 수
        - ratingReviewsTotal:  별점을 매긴 리뷰 수 총개수(2021년 11월 이전 자료들?)

5. restaurant
    - fsasReviews:
        - total : 블로그 리뷰수 총 개수
        - maxItemCount :  ? 뭔지 잘 모르겠음
        - items(리스트로 존재) : 블로그 3개 미리보기 rank순 
            -  url :
            - title:
            - thumbnailUrl
            - date 나온 날짜 
        

In [7]:
len(review_data['review_datas'])

15140

In [8]:
review_lsts = []
review_menu_lsts = []
review_theme_lsts=[]
review_voted_keywords_lsts = []
stats_lsts = []
for num,value in enumerate(review_data['review_datas']):
    for i,v in enumerate(value):
        if i < 2:
            data = v['data']
            reviews = data['visitorReviews']['items']
            #리뷰가 존재 X 
            if len(reviews) == 0:
                print(f'{num}번째 리뷰존재 X')
            else:
                for review in reviews:
                    review_dict = {'original_id': review_data['id'][num]}
                    review_dict['review_id'] = review['id']
                    review_dict['rating'] = review['rating']
                    review_dict['body'] = review['body']
                    review_dict['visited'] =review['visited']
                    votedKeywords = review.get('votedKeywords')
                    if votedKeywords is not None:
                        if len(votedKeywords) != 0:
                            votedKeywords_dict = dict()
                            for keyword in votedKeywords:
                                votedKeywords_dict[keyword['displayName']]= True
                            review_dict['votedKeywords'] = votedKeywords_dict
                    review_lsts.append(review_dict)
                print(f'{num}번째 리뷰들 저장 완료')
        if i == 3:
            stats_dict = dict()
            data = v['data']
            stats= data['visitorReviewStats']
            stats_dict['id'] = stats['id']
            stats_dict['visitorReviewsTotal'] = stats['visitorReviewsTotal']
            stats_dict['ratingReviewsTotal'] = stats['ratingReviewsTotal']
            #리뷰 정보들
            review_stat = stats['review']
            if review_stat is not None:
                stats_dict['avgRating'] = review_stat['avgRating']
                stats_dict['rating_total_count'] = review_stat['totalCount']
                stats_dict['Counts_per_Scores'] = review_stat['scores']
                stats_dict['imageReviewCount'] = review_stat['imageReviewCount']
                stats_dict['authorCount'] = review_stat['authorCount']
                stats_dict['maxSingleReviewScoreCount'] =review_stat['maxSingleReviewScoreCount']

            analysis_stat = stats['analysis']
            if analysis_stat is not None:
                # 테마 언급 횟수
                analysis_themes = analysis_stat['themes']
                tmp_dict1 = {'id': review_data['id'][num]}
                for theme in analysis_themes:
                    tmp_dict1['theme_'+theme['label']] = theme['count']
                review_theme_lsts.append(tmp_dict1)
                
                #메뉴 언급 횟수
                tmp_dict2 =  {'id': review_data['id'][num]}
                analysis_menus = analysis_stat['menus']
                for menu in analysis_menus:
                    tmp_dict2['menus_'+ menu['label']] = menu['count']
                review_menu_lsts.append(tmp_dict2)
                #키워드 투표 
                tmp_dict3 =  {'id': review_data['id'][num]}
                analysis_votedKeywords = analysis_stat['votedKeyword']
                if analysis_votedKeywords is not None:
                    stats_dict['votedKeywords_totalCount'] = analysis_votedKeywords['totalCount']
                    stats_dict['votedKeywords_reviewCount'] = analysis_votedKeywords['reviewCount']
                    stats_dict['votedKeywords_userCount'] = analysis_votedKeywords['userCount']
                    votedKeywords_details =analysis_votedKeywords['details']
                    if len(votedKeywords_details) != 0:
                        for detail in votedKeywords_details:
                            tmp_dict3['votedKeywords_'+ detail['displayName']] = detail['count']
                        review_voted_keywords_lsts.append(tmp_dict3)
            
                stats_lsts.append(stats_dict)
                print(f'{num}번째 리뷰 정보들 저장 완료')


0번째 리뷰들 저장 완료
0번째 리뷰들 저장 완료
0번째 리뷰 정보들 저장 완료
1번째 리뷰들 저장 완료
1번째 리뷰들 저장 완료
1번째 리뷰 정보들 저장 완료
2번째 리뷰들 저장 완료
2번째 리뷰들 저장 완료
2번째 리뷰 정보들 저장 완료
3번째 리뷰들 저장 완료
3번째 리뷰들 저장 완료
3번째 리뷰 정보들 저장 완료
4번째 리뷰들 저장 완료
4번째 리뷰들 저장 완료
4번째 리뷰 정보들 저장 완료
5번째 리뷰들 저장 완료
5번째 리뷰들 저장 완료
5번째 리뷰 정보들 저장 완료
6번째 리뷰들 저장 완료
6번째 리뷰들 저장 완료
6번째 리뷰 정보들 저장 완료
7번째 리뷰들 저장 완료
7번째 리뷰들 저장 완료
7번째 리뷰 정보들 저장 완료
8번째 리뷰들 저장 완료
8번째 리뷰들 저장 완료
8번째 리뷰 정보들 저장 완료
9번째 리뷰들 저장 완료
9번째 리뷰들 저장 완료
9번째 리뷰 정보들 저장 완료
10번째 리뷰존재 X
10번째 리뷰존재 X
10번째 리뷰 정보들 저장 완료
11번째 리뷰들 저장 완료
11번째 리뷰들 저장 완료
11번째 리뷰 정보들 저장 완료
12번째 리뷰들 저장 완료
12번째 리뷰들 저장 완료
12번째 리뷰 정보들 저장 완료
13번째 리뷰들 저장 완료
13번째 리뷰들 저장 완료
13번째 리뷰 정보들 저장 완료
14번째 리뷰들 저장 완료
14번째 리뷰들 저장 완료
14번째 리뷰 정보들 저장 완료
15번째 리뷰들 저장 완료
15번째 리뷰들 저장 완료
15번째 리뷰 정보들 저장 완료
16번째 리뷰들 저장 완료
16번째 리뷰들 저장 완료
16번째 리뷰 정보들 저장 완료
17번째 리뷰들 저장 완료
17번째 리뷰들 저장 완료
17번째 리뷰 정보들 저장 완료
18번째 리뷰들 저장 완료
18번째 리뷰들 저장 완료
18번째 리뷰 정보들 저장 완료
19번째 리뷰존재 X
19번째 리뷰들 저장 완료
19번째 리뷰 정보들 저장 완료
20번째 리뷰들 저장 완료
20번째 리뷰들 저장 완료
20번째 리뷰 정보들 저장 완료
21번째 리뷰존재 X
21번째 리뷰들 저장 완료
21번째

405번째 리뷰들 저장 완료
405번째 리뷰들 저장 완료
405번째 리뷰 정보들 저장 완료
406번째 리뷰존재 X
406번째 리뷰존재 X
406번째 리뷰 정보들 저장 완료
407번째 리뷰들 저장 완료
407번째 리뷰들 저장 완료
407번째 리뷰 정보들 저장 완료
408번째 리뷰존재 X
408번째 리뷰들 저장 완료
408번째 리뷰 정보들 저장 완료
409번째 리뷰들 저장 완료
409번째 리뷰들 저장 완료
409번째 리뷰 정보들 저장 완료
410번째 리뷰들 저장 완료
410번째 리뷰들 저장 완료
410번째 리뷰 정보들 저장 완료
411번째 리뷰존재 X
411번째 리뷰존재 X
412번째 리뷰존재 X
412번째 리뷰존재 X
412번째 리뷰 정보들 저장 완료
413번째 리뷰존재 X
413번째 리뷰존재 X
414번째 리뷰존재 X
414번째 리뷰들 저장 완료
414번째 리뷰 정보들 저장 완료
415번째 리뷰존재 X
415번째 리뷰들 저장 완료
415번째 리뷰 정보들 저장 완료
416번째 리뷰들 저장 완료
416번째 리뷰들 저장 완료
416번째 리뷰 정보들 저장 완료
417번째 리뷰들 저장 완료
417번째 리뷰들 저장 완료
417번째 리뷰 정보들 저장 완료
418번째 리뷰들 저장 완료
418번째 리뷰들 저장 완료
418번째 리뷰 정보들 저장 완료
419번째 리뷰존재 X
419번째 리뷰존재 X
420번째 리뷰들 저장 완료
420번째 리뷰들 저장 완료
420번째 리뷰 정보들 저장 완료
421번째 리뷰존재 X
421번째 리뷰존재 X
421번째 리뷰 정보들 저장 완료
422번째 리뷰들 저장 완료
422번째 리뷰들 저장 완료
422번째 리뷰 정보들 저장 완료
423번째 리뷰들 저장 완료
423번째 리뷰들 저장 완료
423번째 리뷰 정보들 저장 완료
424번째 리뷰존재 X
424번째 리뷰존재 X
424번째 리뷰 정보들 저장 완료
425번째 리뷰존재 X
425번째 리뷰존재 X
425번째 리뷰 정보들 저장 완료
426번째 리뷰존재 X
426번째 리뷰들 저장 완료
426번째 리뷰 정보들 저

664번째 리뷰존재 X
664번째 리뷰 정보들 저장 완료
665번째 리뷰들 저장 완료
665번째 리뷰들 저장 완료
665번째 리뷰 정보들 저장 완료
666번째 리뷰들 저장 완료
666번째 리뷰들 저장 완료
666번째 리뷰 정보들 저장 완료
667번째 리뷰존재 X
667번째 리뷰들 저장 완료
667번째 리뷰 정보들 저장 완료
668번째 리뷰존재 X
668번째 리뷰들 저장 완료
668번째 리뷰 정보들 저장 완료
669번째 리뷰존재 X
669번째 리뷰존재 X
670번째 리뷰존재 X
670번째 리뷰존재 X
671번째 리뷰들 저장 완료
671번째 리뷰들 저장 완료
671번째 리뷰 정보들 저장 완료
672번째 리뷰들 저장 완료
672번째 리뷰들 저장 완료
672번째 리뷰 정보들 저장 완료
673번째 리뷰존재 X
673번째 리뷰들 저장 완료
673번째 리뷰 정보들 저장 완료
674번째 리뷰들 저장 완료
674번째 리뷰들 저장 완료
674번째 리뷰 정보들 저장 완료
675번째 리뷰들 저장 완료
675번째 리뷰들 저장 완료
675번째 리뷰 정보들 저장 완료
676번째 리뷰존재 X
676번째 리뷰들 저장 완료
676번째 리뷰 정보들 저장 완료
677번째 리뷰존재 X
677번째 리뷰존재 X
677번째 리뷰 정보들 저장 완료
678번째 리뷰들 저장 완료
678번째 리뷰들 저장 완료
678번째 리뷰 정보들 저장 완료
679번째 리뷰들 저장 완료
679번째 리뷰들 저장 완료
679번째 리뷰 정보들 저장 완료
680번째 리뷰들 저장 완료
680번째 리뷰들 저장 완료
680번째 리뷰 정보들 저장 완료
681번째 리뷰존재 X
681번째 리뷰존재 X
681번째 리뷰 정보들 저장 완료
682번째 리뷰들 저장 완료
682번째 리뷰들 저장 완료
682번째 리뷰 정보들 저장 완료
683번째 리뷰존재 X
683번째 리뷰들 저장 완료
683번째 리뷰 정보들 저장 완료
684번째 리뷰존재 X
684번째 리뷰들 저장 완료
684번째 리뷰 정보들 저장 완료
685번째 리뷰들 저장 완료
685번째 리뷰들 저장

1021번째 리뷰들 저장 완료
1021번째 리뷰들 저장 완료
1021번째 리뷰 정보들 저장 완료
1022번째 리뷰존재 X
1022번째 리뷰존재 X
1023번째 리뷰들 저장 완료
1023번째 리뷰들 저장 완료
1023번째 리뷰 정보들 저장 완료
1024번째 리뷰들 저장 완료
1024번째 리뷰들 저장 완료
1024번째 리뷰 정보들 저장 완료
1025번째 리뷰들 저장 완료
1025번째 리뷰들 저장 완료
1025번째 리뷰 정보들 저장 완료
1026번째 리뷰들 저장 완료
1026번째 리뷰들 저장 완료
1026번째 리뷰 정보들 저장 완료
1027번째 리뷰존재 X
1027번째 리뷰존재 X
1028번째 리뷰들 저장 완료
1028번째 리뷰들 저장 완료
1028번째 리뷰 정보들 저장 완료
1029번째 리뷰존재 X
1029번째 리뷰존재 X
1029번째 리뷰 정보들 저장 완료
1030번째 리뷰존재 X
1030번째 리뷰존재 X
1031번째 리뷰존재 X
1031번째 리뷰존재 X
1032번째 리뷰존재 X
1032번째 리뷰존재 X
1032번째 리뷰 정보들 저장 완료
1033번째 리뷰들 저장 완료
1033번째 리뷰들 저장 완료
1033번째 리뷰 정보들 저장 완료
1034번째 리뷰들 저장 완료
1034번째 리뷰들 저장 완료
1034번째 리뷰 정보들 저장 완료
1035번째 리뷰들 저장 완료
1035번째 리뷰들 저장 완료
1035번째 리뷰 정보들 저장 완료
1036번째 리뷰존재 X
1036번째 리뷰들 저장 완료
1036번째 리뷰 정보들 저장 완료
1037번째 리뷰존재 X
1037번째 리뷰존재 X
1038번째 리뷰존재 X
1038번째 리뷰존재 X
1038번째 리뷰 정보들 저장 완료
1039번째 리뷰존재 X
1039번째 리뷰존재 X
1040번째 리뷰존재 X
1040번째 리뷰존재 X
1040번째 리뷰 정보들 저장 완료
1041번째 리뷰들 저장 완료
1041번째 리뷰들 저장 완료
1041번째 리뷰 정보들 저장 완료
1042번째 리뷰들 저장 완료
1042번째 리뷰들 저장 완료
1042번째 리뷰 정보들 저

1374번째 리뷰 정보들 저장 완료
1375번째 리뷰들 저장 완료
1375번째 리뷰들 저장 완료
1375번째 리뷰 정보들 저장 완료
1376번째 리뷰들 저장 완료
1376번째 리뷰들 저장 완료
1376번째 리뷰 정보들 저장 완료
1377번째 리뷰들 저장 완료
1377번째 리뷰들 저장 완료
1377번째 리뷰 정보들 저장 완료
1378번째 리뷰존재 X
1378번째 리뷰들 저장 완료
1378번째 리뷰 정보들 저장 완료
1379번째 리뷰존재 X
1379번째 리뷰존재 X
1379번째 리뷰 정보들 저장 완료
1380번째 리뷰들 저장 완료
1380번째 리뷰들 저장 완료
1380번째 리뷰 정보들 저장 완료
1381번째 리뷰존재 X
1381번째 리뷰존재 X
1382번째 리뷰들 저장 완료
1382번째 리뷰들 저장 완료
1382번째 리뷰 정보들 저장 완료
1383번째 리뷰들 저장 완료
1383번째 리뷰들 저장 완료
1383번째 리뷰 정보들 저장 완료
1384번째 리뷰존재 X
1384번째 리뷰존재 X
1385번째 리뷰존재 X
1385번째 리뷰존재 X
1385번째 리뷰 정보들 저장 완료
1386번째 리뷰들 저장 완료
1386번째 리뷰들 저장 완료
1386번째 리뷰 정보들 저장 완료
1387번째 리뷰존재 X
1387번째 리뷰존재 X
1388번째 리뷰들 저장 완료
1388번째 리뷰들 저장 완료
1388번째 리뷰 정보들 저장 완료
1389번째 리뷰존재 X
1389번째 리뷰들 저장 완료
1389번째 리뷰 정보들 저장 완료
1390번째 리뷰존재 X
1390번째 리뷰존재 X
1390번째 리뷰 정보들 저장 완료
1391번째 리뷰들 저장 완료
1391번째 리뷰들 저장 완료
1391번째 리뷰 정보들 저장 완료
1392번째 리뷰존재 X
1392번째 리뷰존재 X
1393번째 리뷰존재 X
1393번째 리뷰들 저장 완료
1393번째 리뷰 정보들 저장 완료
1394번째 리뷰존재 X
1394번째 리뷰존재 X
1395번째 리뷰들 저장 완료
1395번째 리뷰들 저장 완료
1395번째 리뷰 정보들 저장 완료
139

...

13961번째 리뷰 정보들 저장 완료
13962번째 리뷰들 저장 완료
13962번째 리뷰들 저장 완료
13962번째 리뷰 정보들 저장 완료
13963번째 리뷰들 저장 완료
13963번째 리뷰들 저장 완료
13963번째 리뷰 정보들 저장 완료
13964번째 리뷰들 저장 완료
13964번째 리뷰들 저장 완료
13964번째 리뷰 정보들 저장 완료
13965번째 리뷰들 저장 완료
13965번째 리뷰들 저장 완료
13965번째 리뷰 정보들 저장 완료
13966번째 리뷰들 저장 완료
13966번째 리뷰들 저장 완료
13966번째 리뷰 정보들 저장 완료
13967번째 리뷰들 저장 완료
13967번째 리뷰들 저장 완료
13967번째 리뷰 정보들 저장 완료
13968번째 리뷰들 저장 완료
13968번째 리뷰들 저장 완료
13968번째 리뷰 정보들 저장 완료
13969번째 리뷰들 저장 완료
13969번째 리뷰들 저장 완료
13969번째 리뷰 정보들 저장 완료
13970번째 리뷰들 저장 완료
13970번째 리뷰들 저장 완료
13970번째 리뷰 정보들 저장 완료
13971번째 리뷰들 저장 완료
13971번째 리뷰들 저장 완료
13971번째 리뷰 정보들 저장 완료
13972번째 리뷰들 저장 완료
13972번째 리뷰들 저장 완료
13972번째 리뷰 정보들 저장 완료
13973번째 리뷰들 저장 완료
13973번째 리뷰들 저장 완료
13973번째 리뷰 정보들 저장 완료
13974번째 리뷰들 저장 완료
13974번째 리뷰들 저장 완료
13974번째 리뷰 정보들 저장 완료
13975번째 리뷰들 저장 완료
13975번째 리뷰들 저장 완료
13975번째 리뷰 정보들 저장 완료
13976번째 리뷰들 저장 완료
13976번째 리뷰들 저장 완료
13976번째 리뷰 정보들 저장 완료
13977번째 리뷰들 저장 완료
13977번째 리뷰들 저장 완료
13977번째 리뷰 정보들 저장 완료
13978번째 리뷰들 저장 완료
13978번째 리뷰들 저장 완료
13978번째 리뷰 정보들 저장 완료
13979번째 리뷰

14283번째 리뷰존재 X
14283번째 리뷰존재 X
14284번째 리뷰들 저장 완료
14284번째 리뷰들 저장 완료
14284번째 리뷰 정보들 저장 완료
14285번째 리뷰들 저장 완료
14285번째 리뷰들 저장 완료
14285번째 리뷰 정보들 저장 완료
14286번째 리뷰존재 X
14286번째 리뷰존재 X
14286번째 리뷰 정보들 저장 완료
14287번째 리뷰들 저장 완료
14287번째 리뷰들 저장 완료
14287번째 리뷰 정보들 저장 완료
14288번째 리뷰존재 X
14288번째 리뷰존재 X
14288번째 리뷰 정보들 저장 완료
14289번째 리뷰들 저장 완료
14289번째 리뷰들 저장 완료
14289번째 리뷰 정보들 저장 완료
14290번째 리뷰존재 X
14290번째 리뷰들 저장 완료
14290번째 리뷰 정보들 저장 완료
14291번째 리뷰들 저장 완료
14291번째 리뷰들 저장 완료
14291번째 리뷰 정보들 저장 완료
14292번째 리뷰존재 X
14292번째 리뷰들 저장 완료
14292번째 리뷰 정보들 저장 완료
14293번째 리뷰존재 X
14293번째 리뷰들 저장 완료
14293번째 리뷰 정보들 저장 완료
14294번째 리뷰들 저장 완료
14294번째 리뷰들 저장 완료
14294번째 리뷰 정보들 저장 완료
14295번째 리뷰들 저장 완료
14295번째 리뷰들 저장 완료
14295번째 리뷰 정보들 저장 완료
14296번째 리뷰들 저장 완료
14296번째 리뷰들 저장 완료
14296번째 리뷰 정보들 저장 완료
14297번째 리뷰들 저장 완료
14297번째 리뷰들 저장 완료
14297번째 리뷰 정보들 저장 완료
14298번째 리뷰들 저장 완료
14298번째 리뷰들 저장 완료
14298번째 리뷰 정보들 저장 완료
14299번째 리뷰들 저장 완료
14299번째 리뷰들 저장 완료
14299번째 리뷰 정보들 저장 완료
14300번째 리뷰존재 X
14300번째 리뷰존재 X
14300번째 리뷰 정보들 저장 완료
14301번째 리뷰들 저장 완료
14301번째 리뷰

14535번째 리뷰들 저장 완료
14535번째 리뷰들 저장 완료
14535번째 리뷰 정보들 저장 완료
14536번째 리뷰들 저장 완료
14536번째 리뷰들 저장 완료
14536번째 리뷰 정보들 저장 완료
14537번째 리뷰존재 X
14537번째 리뷰존재 X
14537번째 리뷰 정보들 저장 완료
14538번째 리뷰들 저장 완료
14538번째 리뷰들 저장 완료
14538번째 리뷰 정보들 저장 완료
14539번째 리뷰들 저장 완료
14539번째 리뷰들 저장 완료
14539번째 리뷰 정보들 저장 완료
14540번째 리뷰들 저장 완료
14540번째 리뷰들 저장 완료
14540번째 리뷰 정보들 저장 완료
14541번째 리뷰들 저장 완료
14541번째 리뷰들 저장 완료
14541번째 리뷰 정보들 저장 완료
14542번째 리뷰들 저장 완료
14542번째 리뷰들 저장 완료
14542번째 리뷰 정보들 저장 완료
14543번째 리뷰들 저장 완료
14543번째 리뷰들 저장 완료
14543번째 리뷰 정보들 저장 완료
14544번째 리뷰들 저장 완료
14544번째 리뷰들 저장 완료
14544번째 리뷰 정보들 저장 완료
14545번째 리뷰들 저장 완료
14545번째 리뷰들 저장 완료
14545번째 리뷰 정보들 저장 완료
14546번째 리뷰들 저장 완료
14546번째 리뷰들 저장 완료
14546번째 리뷰 정보들 저장 완료
14547번째 리뷰들 저장 완료
14547번째 리뷰들 저장 완료
14547번째 리뷰 정보들 저장 완료
14548번째 리뷰들 저장 완료
14548번째 리뷰들 저장 완료
14548번째 리뷰 정보들 저장 완료
14549번째 리뷰존재 X
14549번째 리뷰존재 X
14549번째 리뷰 정보들 저장 완료
14550번째 리뷰들 저장 완료
14550번째 리뷰들 저장 완료
14550번째 리뷰 정보들 저장 완료
14551번째 리뷰들 저장 완료
14551번째 리뷰들 저장 완료
14551번째 리뷰 정보들 저장 완료
14552번째 리뷰들 저장 완료
14552번째 리뷰들 저장 완료
14552번째

14817번째 리뷰 정보들 저장 완료
14818번째 리뷰들 저장 완료
14818번째 리뷰들 저장 완료
14818번째 리뷰 정보들 저장 완료
14819번째 리뷰들 저장 완료
14819번째 리뷰들 저장 완료
14819번째 리뷰 정보들 저장 완료
14820번째 리뷰존재 X
14820번째 리뷰존재 X
14820번째 리뷰 정보들 저장 완료
14821번째 리뷰들 저장 완료
14821번째 리뷰들 저장 완료
14821번째 리뷰 정보들 저장 완료
14822번째 리뷰존재 X
14822번째 리뷰들 저장 완료
14822번째 리뷰 정보들 저장 완료
14823번째 리뷰들 저장 완료
14823번째 리뷰들 저장 완료
14823번째 리뷰 정보들 저장 완료
14824번째 리뷰들 저장 완료
14824번째 리뷰들 저장 완료
14824번째 리뷰 정보들 저장 완료
14825번째 리뷰들 저장 완료
14825번째 리뷰들 저장 완료
14825번째 리뷰 정보들 저장 완료
14826번째 리뷰들 저장 완료
14826번째 리뷰들 저장 완료
14826번째 리뷰 정보들 저장 완료
14827번째 리뷰들 저장 완료
14827번째 리뷰들 저장 완료
14827번째 리뷰 정보들 저장 완료
14828번째 리뷰들 저장 완료
14828번째 리뷰들 저장 완료
14828번째 리뷰 정보들 저장 완료
14829번째 리뷰들 저장 완료
14829번째 리뷰들 저장 완료
14829번째 리뷰 정보들 저장 완료
14830번째 리뷰들 저장 완료
14830번째 리뷰들 저장 완료
14830번째 리뷰 정보들 저장 완료
14831번째 리뷰들 저장 완료
14831번째 리뷰들 저장 완료
14831번째 리뷰 정보들 저장 완료
14832번째 리뷰존재 X
14832번째 리뷰존재 X
14832번째 리뷰 정보들 저장 완료
14833번째 리뷰들 저장 완료
14833번째 리뷰들 저장 완료
14833번째 리뷰 정보들 저장 완료
14834번째 리뷰들 저장 완료
14834번째 리뷰들 저장 완료
14834번째 리뷰 정보들 저장 완료
14835번째 리뷰들 저장 완료
14835번째

15135번째 리뷰들 저장 완료
15135번째 리뷰들 저장 완료
15135번째 리뷰 정보들 저장 완료
15136번째 리뷰들 저장 완료
15136번째 리뷰들 저장 완료
15136번째 리뷰 정보들 저장 완료
15137번째 리뷰들 저장 완료
15137번째 리뷰들 저장 완료
15137번째 리뷰 정보들 저장 완료
15138번째 리뷰들 저장 완료
15138번째 리뷰들 저장 완료
15138번째 리뷰 정보들 저장 완료
15139번째 리뷰들 저장 완료
15139번째 리뷰들 저장 완료
15139번째 리뷰 정보들 저장 완료


In [9]:
review_df =pd.DataFrame(review_lsts)
review_stats_df =pd.DataFrame(stats_lsts)
review_menus_df = pd.DataFrame(review_menu_lsts)
review_theme_df = pd.DataFrame(review_theme_lsts)
review_voted_keywords_df = pd.DataFrame(review_voted_keywords_lsts)

In [10]:
review_menus_df.head()

,id,menus_게장,menus_간장게장,menus_볶음밥,menus_매운탕,menus_전복,menus_튀김,menus_고등어,menus_고등어회,menus_갈치,...,menus_마블케이크,menus_레몬타르트,menus_마카롱아이스크림,menus_풀드포크샌드위치,menus_멸치찌개,menus_체다치즈피자,menus_번데기탕,menus_생오리로스,menus_기스면,menus_스모크치킨
0,12883219,238.0,177.0,86.0,66.0,62.0,50.0,43.0,36.0,36.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,1207652081,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,35269176,3.0,3.0,NaN,NaN,2.0,NaN,NaN,NaN,3.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,11710933,NaN,NaN,2.0,3.0,6.0,3.0,2.0,NaN,3.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,1480037450,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [11]:
review_theme_df.head()

,id,theme_만족도,theme_맛,theme_서비스,theme_음식량,theme_위치,theme_목적,theme_예약,theme_가격,theme_전망,...,theme_입장,theme_사용성,theme_즐길거리,theme_시설물관리,theme_사용감,theme_건강,theme_디자인,theme_포장,theme_인기도,theme_용량
0,12883219,424.0,388.0,259.0,59.0,48.0,43.0,33.0,32.0,24.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,1207652081,13.0,12.0,8.0,1.0,NaN,1.0,NaN,3.0,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,35269176,9.0,10.0,3.0,1.0,NaN,NaN,NaN,1.0,1.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,11710933,39.0,41.0,19.0,7.0,1.0,3.0,NaN,4.0,6.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,1480037450,27.0,38.0,24.0,3.0,NaN,3.0,NaN,1.0,1.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [12]:
review_menus_df.shape,review_theme_df.shape,review_voted_keywords_df.shape

((13599, 2672), (13599, 48), (12021, 83))

In [13]:
review

{'id': '625172e93161db0050bb381a',
 'rating': None,
 'author': {'id': '2hwuN',
  'nickname': '수원댁42',
  'from': 'NAVER',
  'imageUrl': None,
  'objectId': '5c780118f18d2007eb1a4c22',
  'url': 'https://m.place.naver.com/my/5c780118f18d2007eb1a4c22/review?v=2',
  'review': None,
  'theme': None,
  '__typename': 'VisitorReviewAuthor'},
 'body': '깔끔하고 맛있어요',
 'thumbnail': None,
 'media': [],
 'tags': None,
 'status': 'published',
 'visitCount': 1,
 'viewCount': 1810,
 'visited': '4.7.목',
 'created': '4.9.토',
 'reply': {'editUrl': 'https://new-m.smartplace.naver.com/bizes/place-id/16964053/reviews/625172e93161db0050bb381a',
  'body': None,
  'editedBy': None,
  'created': None,
  'replyTitle': None,
  '__typename': 'VisitorReviewReply'},
 'originType': '영수증',
 'item': None,
 'isFollowing': None,
 'language': 'ko',
 'highlightOffsets': None,
 'translatedText': None,
 'businessName': '향촌마을',
 'receiptInfoUrl': 'https://m.place.naver.com/my/review/625172e93161db0050bb381a/reply#showReceipt',
 

In [14]:
review_voted_keywords_df.head()

,id,votedKeywords_음식이 맛있어요,votedKeywords_재료가 신선해요,votedKeywords_친절해요,votedKeywords_뷰가 좋아요,votedKeywords_가성비가 좋아요,votedKeywords_매장이 넓어요,votedKeywords_양이 많아요,votedKeywords_단체모임 하기 좋아요,votedKeywords_특별한 메뉴가 있어요,...,votedKeywords_과채가 신선해요,votedKeywords_즉석조리 식품이 맛있어요,votedKeywords_상품이 다양해요,votedKeywords_신상품이 많아요,votedKeywords_향신료가 강하지 않아요,votedKeywords_읽을만한 책이 많아요,votedKeywords_라이브공연이 훌륭해요,votedKeywords_포장이 정성스러워요,votedKeywords_추천을 잘해줘요,votedKeywords_설명이 자세해요
0,12883219,114.0,56.0,49.0,49.0,35.0,35.0,33.0,21.0,20.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,1207652081,17.0,13.0,7.0,NaN,14.0,1.0,4.0,3.0,2.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,35269176,41.0,23.0,18.0,5.0,9.0,4.0,10.0,10.0,6.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,11710933,23.0,12.0,17.0,15.0,3.0,7.0,2.0,5.0,4.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,1480037450,30.0,9.0,16.0,13.0,4.0,2.0,NaN,2.0,2.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [15]:
review_df.head(24)

,original_id,review_id,rating,body,visited,votedKeywords
0,12883219,62bb7a3df3310e76019dee03,NaN,,6.28.화,"{'음식이 맛있어요': True, '매장이 넓어요': True, '친절해요': Tr..."
1,12883219,629ef153e003fa8ab054a713,NaN,바다 뷰도 너무 좋고 점심특선 회덮밥을 시켰더니 매운탕이 서비스로 나오고 가성비가 ...,6.7.화,"{'음식이 맛있어요': True, '양이 많아요': True, '가성비가 좋아요':..."
2,12883219,629eefc9e7f2c886b0b0760d,NaN,바다 앞에 위치한 식당인데 참 가성비가 좋아요 가격 대비 반찬들 포함해서 푸짐합니다...,6.7.화,"{'음식이 맛있어요': True, '양이 많아요': True, '재료가 신선해요':..."
3,12883219,629c803a6a9dd26d326d6d57,NaN,찌깨다시가 딱 젓가락갈 것만 올려주세요. 그리고 게장 아주 맛깔납니다. 담에는 게장...,6.4.토,"{'음식이 맛있어요': True, '재료가 신선해요': True, '양이 많아요':..."
4,12883219,629cb351ea57d4bd9c704b6a,NaN,점심특선으로 회덮밥 주문했는데 매운탕까지 나와요!! 기본반찬도 너무 잘나와요!!,6.4.토,"{'음식이 맛있어요': True, '재료가 신선해요': True, '양이 많아요':..."
5,12883219,62988b6f5621193b773a6b14,NaN,,6.2.목,"{'음식이 맛있어요': True, '양이 많아요': True, '매장이 넓어요': ..."
6,12883219,629192a280856178aa64f10d,NaN,삼다도횟집은 도두봉과 가까워요~점심특선 가성비최고! 맛도 뷰도 최고였어요,5.26.목,"{'음식이 맛있어요': True, '뷰가 좋아요': True}"
7,12883219,62846deb751bd8fa4615d485,NaN,잘먹엇습니다!,5.18.수,"{'양이 많아요': True, '매장이 넓어요': True, '친절해요': True}"
8,12883219,62819fec8dd0e549bef48fba,NaN,초밥이랑 회덮밭 회도 실하고 무엇보다 바로 앞 해안도로 뷰도 좋네요. 기본으로 매운...,5.15.일,"{'음식이 맛있어요': True, '가성비가 좋아요': True, '매장이 넓어요'..."
9,12883219,6261821fba7d0e005155a095,NaN,제주 여행 올때마다 들를거같은 곳\n고사리나물과 김치가 특별나게 맛있음\n맛있고 친...,4.22.금,"{'뷰가 좋아요': True, '매장이 넓어요': True, '양이 많아요': Tr..."


In [16]:
review_df.shape

(140756, 6)

In [17]:
review_df.to_csv('./review_body_preprocessed.csv',index=False)

In [18]:
review_stats_df.to_csv('./review_stats_preprocessed.csv',index=False)
review_menus_df.to_csv('./review_menus_preprocessed.csv',index=False),
review_theme_df.to_csv('./review_themes_preprocessed.csv',index=False)
review_voted_keywords_df.to_csv('./review_voted_keywords_preprocessed.csv',index=False)

In [19]:
pd.read_csv('./review_body_preprocessed.csv')

,original_id,review_id,rating,body,visited,votedKeywords
0,12883219,62bb7a3df3310e76019dee03,NaN,NaN,6.28.화,"{'음식이 맛있어요': True, '매장이 넓어요': True, '친절해요': Tr..."
1,12883219,629ef153e003fa8ab054a713,NaN,바다 뷰도 너무 좋고 점심특선 회덮밥을 시켰더니 매운탕이 서비스로 나오고 가성비가 ...,6.7.화,"{'음식이 맛있어요': True, '양이 많아요': True, '가성비가 좋아요':..."
2,12883219,629eefc9e7f2c886b0b0760d,NaN,바다 앞에 위치한 식당인데 참 가성비가 좋아요 가격 대비 반찬들 포함해서 푸짐합니다...,6.7.화,"{'음식이 맛있어요': True, '양이 많아요': True, '재료가 신선해요':..."
3,12883219,629c803a6a9dd26d326d6d57,NaN,찌깨다시가 딱 젓가락갈 것만 올려주세요. 그리고 게장 아주 맛깔납니다. 담에는 게장...,6.4.토,"{'음식이 맛있어요': True, '재료가 신선해요': True, '양이 많아요':..."
4,12883219,629cb351ea57d4bd9c704b6a,NaN,점심특선으로 회덮밥 주문했는데 매운탕까지 나와요!! 기본반찬도 너무 잘나와요!!,6.4.토,"{'음식이 맛있어요': True, '재료가 신선해요': True, '양이 많아요':..."
...,...,...,...,...,...,...
140751,16964053,601d54c1b3cdd300613ec9d2,5.0,오름 가서 열심히 걷고 먹었더니 정말 허겁지겁 맛있게 먹었네요,20.12.22.화,NaN
140752,16964053,5dccaf5893237900102a6cc1,4.0,맛있습니다.\n아주 조아요,19.11.12.화,NaN
140753,16964053,62a8193cef208f71d4fb76f2,NaN,음식이 깔끔.정갈하고 맛있어요. 개인적으로 타 한식뷔페에서 계란후라이 라면 등 셀프...,6.14.화,NaN
140754,16964053,6226c55a2d221d00457f8c51,NaN,음식도 다양하고 맛도 좋아요~,3.8.화,NaN
